In [1]:
from chat import ContextManager
context_manager = ContextManager()
context_manager.load_from_file("./twilio/context.txt")

In [2]:
def test(agent):
    with open("transcript.txt", "r") as file_handler:
        transcript = file_handler.read().splitlines()
    
    for customer_support_response in transcript:
        print("Customer support:", customer_support_response)
        print("Agent response:", agent(customer_support_response))

In [3]:
from chat_agents import ContextAgent, EfficientAgent, EfficientContextAgent, SystemBasedAgent, CookBook

In [4]:
agent = EfficientContextAgent("create a new account", "People's Gas", context_manager)

In [6]:
test(agent)

Customer support: Hi! This is People's gas, how could we help you today?
Agent response: Hi there, I'd like to create a new account with People's gas, please.
Customer support: What is your name?
Agent response: My name is Ekrem Korkmaz.
Customer support: How do you spell that?
Agent response: E-K-R-E-M   K-O-R-K-M-A-Z.
Customer support: What is your date of birth?
Agent response: My date of birth is 03/02/2001.
Customer support: What is your social security number?
Agent response: My SSN is 123-45-678.
Customer support: Do you already have a People's account?
Agent response: No, this is my first time creating an account with People's Gas.
Customer support: What could we help you with today?
Agent response: Hi! I'd like to create a new account with People's Gas, please.
Customer support: What is your home address?
Agent response: My home address is 5032 Forbes Ave, SMC 1000, Pittsburgh PA 15289.
Customer support: Could you repeat that?
Agent response: 5032 Forbes Ave, SMC 1000, Pittsbu

In [19]:
import openai

class SummaryAgent():
    def __init__(self):
        self.model = "gpt-3.5-turbo" 
        self.generate_agent_description()
        self.agent_description = {"role": "system", "content": self.agent_description_prompt}
    
    def generate_agent_description(self):
        self.agent_description_prompt = f"""
Use the provided conversations delimited by triple quotes to answer questions.
You're given a conversation between a customer and a customer service representative.
Your goal is to summarize the conversation to actionable items. 
        """
        
    def engineer_prompt(self, dialogue_history):
        dialogue = ""
        for dialogue_dict in dialogue_history:
            if dialogue_dict["role"] == "user":
                dialogue += "\n" + dialogue_dict["content"]
            elif dialogue_dict["role"] == "assistant":
                dialogue += "\n" + "Human" + dialogue_dict["content"]
        
        prompt = f"""
You are a personal assistant. You're listening to a conversation between a human and a customer service representative. This dialogue is given between the triple quotes. Summarize the following dialogue to a list of important information provided by the customer service representative. This information shouldn't include anything the human would know before the conversation. Your response should be brief.
\"\"\"
{dialogue}
\"\"\"
"""
        return prompt
    
    def __call__(self, agent):
        dialogue_history = agent.dialogue_history
        
        prompt = self.engineer_prompt(dialogue_history)
        messages = [self.agent_description, {"role": "user", "content": prompt}]
        completion = openai.ChatCompletion.create(
            model=self.model,
            messages=messages
        )
        response = completion.choices[0].message
        
        return response.content

In [20]:
summary = SummaryAgent()

In [21]:
summary(agent)

"- Ekrem Korkmaz wants to create a new account with People's Gas.\n- The customer service agent asks for his personal information such as name, date of birth, social security number, home address, and email address.\n- This is his first time creating an account with People's Gas.\n- The customer service agent assigns a new account number, PG-345-67.\n- Two gas operators will come to the customer's house to set up the new account tomorrow between 8 am and noon."